# A-gs model and implementation (simulation CO2 and H2O flux)

## Initialize data and model

### Setup and fetch data

In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2001,2021)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath   = os.path.join('../')
print('datapath is set to %s'%datapath)

# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install plotly 
# !pip install cufflinks
#!pip install colorspacious
#!pip install seaborn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import scipy.stats as stats
#cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile

from Ags_model import runAgs, calc_LE

In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
if not 'progress' in globals(): progress = list()
if not 'dataloaded' in progress:
  # Read files
    df_EC           = Read_LoobosEddFinal    (years,datapath)
    df_Stor         = Read_LooStor           (years,datapath)
    df_Comb         = Read_LoodatGapfill     (years,datapath)
    df_NEE          = Read_Loobos_halfhourly (years,datapath)
    df_meteo        = Read_Loobos_meteo      (years,datapath)
    df_soil         = Read_Loobos_soil       (years,datapath) 
    df_profile      = Read_Loobos_profile    (years,datapath)
    progress.append('dataloaded')

In [ ]:
# Make filter for GPP orginial data and not gapfilled
#General filters
I = ((df_Comb['GPP_fqc']==0)&(df_meteo['PAR']>0))
#t = df_profile.index                                          
#time = (t < np.datetime64('2013-05-08')) | (t > np.datetime64('2013-06-01'))

# Filter for CO2 data
CO2 = (df_profile['CO2level1'] > 300)

# Filter for L(o)corr data
Locorr= (df_meteo['L(o)corr']>0) 

# Filter for VPD data
VPD = (df_Comb['VPD']>=0)

# Filter for U-star
Ustar = (df_EC['U-star']>=0)

# Combine all filters
filter = I & CO2 & Locorr & VPD & Ustar

#Column 'CO2' is input from df_profile
#df_profile_CO2 = df_profile[CO2]
#df_profile_filter = df_profile_CO2[I]
df_profile_filter = df_profile[filter]

#Column 'L(o)corr' and 'PAR' are inputs from df_meteo
#df_meteo_CO2 = df_meteo[CO2]
#df_meteo_filter = df_meteo_CO2[I]
df_meteo_filter = df_meteo[filter]

#Columns 'VPD' and 'Tair' are inputs from df_Comb
#df_Comb_CO2 = df_Comb[CO2]
#df_Comb_filter = df_Comb_CO2[I]
df_Comb_filter = df_Comb[filter]

# Columns 'Mea_Windsp' and 'U-star' are inputs from df_EC
#df_EC_CO2 = df_EC[CO2]
#df_EC_filter = df_EC_CO2[I]
df_EC_filter = df_EC[filter]

In [ ]:
#additional filters for L(o)
df_meteo_filter['L(o)'] = df_meteo_filter['L(o)'].loc[df_meteo_filter['L(o)']>-50]
df_meteo_filter['L(o)'] = df_meteo_filter['L(o)'].loc[df_meteo_filter['L(o)']<20]

In [ ]:
#additional filters for L(o)
df_meteo['L(o)'] = df_meteo['L(o)'].loc[df_meteo['L(o)']>-50]
df_meteo['L(o)'] = df_meteo['L(o)'].loc[df_meteo['L(o)']<20]

In [ ]:
#additional filters for Te-L(o)
df_meteo['Te-L(o)'] = df_meteo['Te-L(o)'].loc[df_meteo['Te-L(o)']<40]

### Run A-gs model

In [ ]:
# Run A-gs reference, unaltered
an_final_10,an_umol_10,rs_10, ra = runAgs(df_profile_filter,df_Comb_filter,df_meteo_filter,df_EC_filter,fstr=1.0)

In [ ]:
# Run A-gs model for the CO2 simulation

df_profile_filter2 = df_profile_filter.copy()
df_profile_filter2['CO2level1']=df_profile_filter2['CO2level1']*1.1236

an_final_CO2,an_umol_CO2,rs_CO2, ra_CO2 = runAgs(df_profile_filter2,df_Comb_filter,df_meteo_filter,df_EC_filter,fstr=1.0)

In [ ]:
# Run A-gs model for the VPD simulation

df_Comb_filter2 = df_Comb_filter.copy()
df_Comb_filter2['VPD'] = df_Comb_filter2['VPD']*1.7109 

an_final_VPD,an_umol_VPD,rs_VPD, ra_VPD = runAgs(df_profile_filter,df_Comb_filter2,df_meteo_filter,df_EC_filter,fstr=1.0)

In [ ]:
# Run A-gs model for the PAR simulation

df_meteo_filter2 = df_meteo_filter.copy()
df_meteo_filter2['PAR'] = df_meteo_filter2['PAR']*0.9985

an_final_PAR,an_umol_PAR,rs_PAR, ra_PAR = runAgs(df_profile_filter,df_Comb_filter,df_meteo_filter2,df_EC_filter,fstr=1.0)

In [ ]:
# Run A-gs model for the Lout simulation

df_meteo_filter2 = df_meteo_filter.copy()
df_meteo_filter2['L(o)corr'] = df_meteo_filter2['L(o)corr']*1.0182

an_final_LO,an_umol_LO,rs_LO, ra_LO = runAgs(df_profile_filter,df_Comb_filter,df_meteo_filter2,df_EC_filter,fstr=1.0)

## Calcuate ET

### Assemble dataframe 'df_ET' that will hold output and fill with inputs

In [ ]:
def init_ETframe(rs_series):
    df_ET = pd.concat([df_meteo['L(o)'],df_meteo['Te-L(o)'],df_profile['Pressure'],df_Comb['VPD'],df_Comb['rH'],df_meteo['P(mast)']],axis=1,sort=False)
    #convert Pressure from hPa to kPa 
    df_ET['p_kPa']=df_ET['Pressure']/10
    df_ET['VPD_adj']=df_ET['VPD'].loc[df_ET['VPD']>0] #some outlier values for VPD are negative, remove from dataset
    df_ET['VPD_adj']=df_ET['VPD_adj']/10  # VPD from df_Comb is in hPa, I need kPa, so hPa/10 = kPa
    df_ET['rs']=rs_series.to_frame(name="rs")
    df_ET['ra']=ra.to_frame(name="ra")
    return df_ET

In [ ]:
df_ET_10=init_ETframe(rs_10)

In [ ]:
df_ET_10=calc_LE(df_ET_10)

### sens analsis

In [ ]:
#start='2001-05-01 00:00'
#end='2020-08-30 00:00'

def plot_sens_experiment_GPP(ref_input , modified_input, var_name):

    df_data = pd.DataFrame()
    df_data['ref_data'] = ref_input
    df_data = df_data.merge(modified_input, how='inner', left_index=True, right_index=True)
    df_data = df_data.loc[(df_data.index.month>=4) & (df_data.index.month<=9)]
    df_data = df_data.dropna()

    slope, intercept, r_value, p_value, std_err = stats.linregress(df_data['ref_data'], df_data['modified_data'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
    print('slope, intercept, R2:', slope, intercept, r_value**2)

    fig,ax = plt.subplots()
    p1=ax.scatter(df_data['ref_data'],df_data['modified_data'],marker='o',s=20,c=df_data.index.hour,cmap='viridis') #scatter(x,y)
    ax.axline((0.0,intercept),slope=slope,c='black',label='slope')
    ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')
    ax.set_ylim(0,40)
    ax.set_xlim(0,40)
    ax.set_xlabel(r'An (reference run) [$\mu molm^{-2}s^{-1}$]')

    label_strings={'VPD':r'An (VPD 171.09% run) [$\mu molm^{-2}s^{-1}$]',
    'CO2':r'An (CO$_2$ 112.36% run) [$\mu molm^{-2}s^{-1}$]',
    'LO':r'An (L$_{out}$ 101.82% run) [$\mu molm^{-2}s^{-1}$]',
    'PAR':r'An (PAR 99.85% run) [$\mu molm^{-2}s^{-1}$]'}

    title_strings={'VPD':'VPD',
                   'CO2':'CO_{2}',
                   'LO':'L_{out}',
                   'PAR':'PAR'}
            
    
    ax.set_ylabel(label_strings[var_name])
    fig.suptitle(r'Correlation of increased ${}$ to reference run for An,'.format(title_strings[var_name])+'\n (2001-2019 during growth seasons May-Sep.)')
    ax.set_title(r'intercept = {:.3f}, slope = {:.3f}, $R^2$ = {:.4f}'.format(intercept,slope,r_value**2))
    ax.legend(loc='upper left')
    fig.subplots_adjust(top=0.86) #fix the top margin text overlap  
    cbar=plt.colorbar(p1,ax=ax)
    cbar.ax.set_ylabel('Hour')

### CO2

In [ ]:
an_umol_10.rename('ref_data',inplace=True)
an_umol_CO2.rename('modified_data',inplace=True)
plot_sens_experiment_GPP(an_umol_10 , an_umol_CO2, 'CO2')

### PAR

In [ ]:
an_umol_10.rename('ref_data',inplace=True)
an_umol_PAR.rename('modified_data',inplace=True)
plot_sens_experiment_GPP(an_umol_10 , an_umol_PAR, 'PAR')

### VPD

In [ ]:
an_umol_10.rename('ref_data',inplace=True)
an_umol_VPD.rename('modified_data',inplace=True)
plot_sens_experiment_GPP(an_umol_10 , an_umol_VPD, 'VPD')

### L(out)

In [ ]:
an_umol_10.rename('ref_data',inplace=True)
an_umol_LO.rename('modified_data',inplace=True)
plot_sens_experiment_GPP(an_umol_10 , an_umol_LO, 'LO')